# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [88]:
# import libraries
import os
import re
import numpy as np
import pandas as pd
import pickle

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, precision_score, recall_score, \
                            f1_score, accuracy_score

time: 4.09 ms (started: 2021-01-03 19:10:20 +00:00)


In [1]:
# import & download nltk-library
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k4t\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\k4t\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k4t\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
# installing execution time measurement tools 
!pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2021-01-03 21:44:54 +01:00)


In [62]:
# load data from database
engine = create_engine('sqlite:///db/disaster_messages_db.db')

sqlquery = "SELECT * FROM disaster_messages;"
df = pd.read_sql(sqlquery, con=engine)
df.head()

,id,message,original,genre,category:related,category:request,category:offer,category:aid_related,category:medical_help,category:medical_products,...,category:aid_centers,category:other_infrastructure,category:weather_related,category:floods,category:storm,category:fire,category:earthquake,category:cold,category:other_weather,category:direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


time: 635 ms (started: 2021-01-03 18:47:59 +00:00)


In [63]:
# splitting dataset into X (feature-data) and Y (label-data)
X = df['message']
Y = df.iloc[:, -36:] # the last 36(categories)-column
print(f'X-shape {X.shape} and Y-shape {Y.shape}')

X-shape (26216,) and Y-shape (26216, 36)
time: 7.9 ms (started: 2021-01-03 18:47:59 +00:00)


### 2. Write a tokenization function to process your text data

In [64]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text: str, url_regex:str = url_regex):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'placeholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

time: 6.18 ms (started: 2021-01-03 18:47:59 +00:00)


In [65]:
X[1]

'Is the Hurricane over or is it not over'

time: 9.57 ms (started: 2021-01-03 18:47:59 +00:00)


In [66]:
tokenize(X[1])

['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']

time: 6.58 ms (started: 2021-01-03 18:47:59 +00:00)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [67]:
rfc_pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('rfc_classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

time: 2.43 ms (started: 2021-01-03 18:47:59 +00:00)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [68]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

time: 31.2 ms (started: 2021-01-03 18:47:59 +00:00)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [69]:
rfc_pipeline.fit(X_train, Y_train)
Y_pred = rfc_pipeline.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

time: 1min 18s (started: 2021-01-03 18:47:59 +00:00)


In [70]:
def display_results(model, Y_test, Y_pred):
    accuracy = (Y_pred == Y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

time: 2.34 ms (started: 2021-01-03 18:49:18 +00:00)


In [71]:
display_results(rfc_pipeline, Y_test, Y_pred)

The model accuracy is 0.942
time: 5.81 ms (started: 2021-01-03 18:49:18 +00:00)


### 6. Improve your model
Use grid search to find better parameters. 

In [81]:
list(rfc_pipeline.get_params().keys())

['memory',
 'steps',
 'verbose',
 'text_pipeline',
 'rfc_classifier',
 'text_pipeline__memory',
 'text_pipeline__steps',
 'text_pipeline__verbose',
 'text_pipeline__vect',
 'text_pipeline__tfidf',
 'text_pipeline__vect__analyzer',
 'text_pipeline__vect__binary',
 'text_pipeline__vect__decode_error',
 'text_pipeline__vect__dtype',
 'text_pipeline__vect__encoding',
 'text_pipeline__vect__input',
 'text_pipeline__vect__lowercase',
 'text_pipeline__vect__max_df',
 'text_pipeline__vect__max_features',
 'text_pipeline__vect__min_df',
 'text_pipeline__vect__ngram_range',
 'text_pipeline__vect__preprocessor',
 'text_pipeline__vect__stop_words',
 'text_pipeline__vect__strip_accents',
 'text_pipeline__vect__token_pattern',
 'text_pipeline__vect__tokenizer',
 'text_pipeline__vect__vocabulary',
 'text_pipeline__tfidf__norm',
 'text_pipeline__tfidf__smooth_idf',
 'text_pipeline__tfidf__sublinear_tf',
 'text_pipeline__tfidf__use_idf',
 'rfc_classifier__estimator__bootstrap',
 'rfc_classifier__estima

time: 7.1 ms (started: 2021-01-03 18:51:40 +00:00)


In [83]:
parameters = {
        'text_pipeline__tfidf__use_idf': (True, False),
        'rfc_classifier__estimator__n_estimators': [10, 20]
}
cv = GridSearchCV(rfc_pipeline, param_grid=parameters)
cv

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('text_pipeline',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         CountVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.int64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                     

time: 40.3 ms (started: 2021-01-03 18:51:49 +00:00)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [84]:
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


time: 14min 22s (started: 2021-01-03 18:51:51 +00:00)


In [85]:
display_results(rfc_pipeline, Y_test, Y_pred)

The model accuracy is 0.944
time: 5.53 ms (started: 2021-01-03 19:06:13 +00:00)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

=> some Classifier Idea:
- AdaBoostClassifier : AdaBoostClassifier()
- KNN Classifier : 
- SVM Classifier : SVC()
- Naive Bayes Classifier : MultinomialNB()

In [ ]:
# TODO Refactor code
# TODO Iterate with classifier possibility

### 9. Export your model as a pickle file

In [89]:
model_base_folder = 'models'
if not os.path.exists(model_base_folder):
    os.makedirs(model_base_folder)

# Create a pickle file for the model
file_name = f'{model_base_folder}/model.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

time: 3.74 s (started: 2021-01-03 19:10:23 +00:00)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# see `models/train_classifier.py`
